In [1]:
# First, we run SPIN's demo.py inference on an (cropped) image from 3dpw

img_path = 'examples/image_00502.jpg'
#img_path = 'examples/image_00980.jpg'

pickle_path = 'data/3dpw/sequenceFiles/validation/courtyard_basketball_01.pkl'
#pickle_path = 'data/3dpw/sequenceFiles/validation/outdoors_parcours_01.pkl'

frame = 502

In [2]:
"""
Demo code

To run our method, you need a bounding box around the person. The person needs to be centered inside the bounding box and the bounding box should be relatively tight. You can either supply the bounding box directly or provide an [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) detection file. In the latter case we infer the bounding box from the detections.

In summary, we provide 3 different ways to use our demo code and models:
1. Provide only an input image (using ```--img```), in which case it is assumed that it is already cropped with the person centered in the image.
2. Provide an input image as before, together with the OpenPose detection .json (using ```--openpose```). Our code will use the detections to compute the bounding box and crop the image.
3. Provide an image and a bounding box (using ```--bbox```). The expected format for the json file can be seen in ```examples/im1010_bbox.json```.

Example with OpenPose detection .json
```
python3 demo.py --checkpoint=data/model_checkpoint.pt --img=examples/im1010.png --openpose=examples/im1010_openpose.json
```
Example with predefined Bounding Box
```
python3 demo.py --checkpoint=data/model_checkpoint.pt --img=examples/im1010.png --bbox=examples/im1010_bbox.json
```
Example with cropped and centered image
```
python3 demo.py --checkpoint=data/model_checkpoint.pt --img=examples/im1010.png
```

Running the previous command will save the results in ```examples/im1010_{shape,shape_side}.png```. The file ```im1010_shape.png``` shows the overlayed reconstruction of human shape. We also render a side view, saved in ```im1010_shape_side.png```.
"""

import torch
from torchvision.transforms import Normalize
import numpy as np
import cv2
import argparse
import json

from models import hmr, SMPL
from utils.imutils import crop
from utils.renderer import Renderer
import config
import constants

parser = argparse.ArgumentParser()
parser.add_argument('--checkpoint', required=True, help='Path to pretrained checkpoint')
parser.add_argument('--img', type=str, required=True, help='Path to input image')
parser.add_argument('--bbox', type=str, default=None, help='Path to .json file containing bounding box coordinates')
parser.add_argument('--openpose', type=str, default=None, help='Path to .json containing openpose detections')
parser.add_argument('--outfile', type=str, default=None, help='Filename of output images. If not set use input filename.')

def bbox_from_openpose(openpose_file, rescale=1.2, detection_thresh=0.2):
    """Get center and scale for bounding box from openpose detections."""
    with open(openpose_file, 'r') as f:
        keypoints = json.load(f)['people'][0]['pose_keypoints_2d']
    keypoints = np.reshape(np.array(keypoints), (-1,3))
    valid = keypoints[:,-1] > detection_thresh
    valid_keypoints = keypoints[valid][:,:-1]
    center = valid_keypoints.mean(axis=0)
    bbox_size = (valid_keypoints.max(axis=0) - valid_keypoints.min(axis=0)).max()
    # adjust bounding box tightness
    scale = bbox_size / 200.0
    scale *= rescale
    return center, scale

def bbox_from_json(bbox_file):
    """Get center and scale of bounding box from bounding box annotations.
    The expected format is [top_left(x), top_left(y), width, height].
    """
    with open(bbox_file, 'r') as f:
        bbox = np.array(json.load(f)['bbox']).astype(np.float32)
    ul_corner = bbox[:2]
    center = ul_corner + 0.5 * bbox[2:]
    width = max(bbox[2], bbox[3])
    scale = width / 200.0
    # make sure the bounding box is rectangular
    return center, scale

def process_image(img_file, bbox_file, openpose_file, input_res=224):
    """Read image, do preprocessing and possibly crop it according to the bounding box.
    If there are bounding box annotations, use them to crop the image.
    If no bounding box is specified but openpose detections are available, use them to get the bounding box.
    """
    normalize_img = Normalize(mean=constants.IMG_NORM_MEAN, std=constants.IMG_NORM_STD)
    img = cv2.imread(img_file)[:,:,::-1].copy() # PyTorch does not support negative stride at the moment
    if bbox_file is None and openpose_file is None:
        # Assume that the person is centerered in the image
        height = img.shape[0]
        width = img.shape[1]
        center = np.array([width // 2, height // 2])
        scale = max(height, width) / 200
    else:
        if bbox_file is not None:
            center, scale = bbox_from_json(bbox_file)
        elif openpose_file is not None:
            center, scale = bbox_from_openpose(openpose_file)
    img = crop(img, center, scale, (input_res, input_res))
    img = img.astype(np.float32) / 255.
    img = torch.from_numpy(img).permute(2,0,1)
    norm_img = normalize_img(img.clone())[None]
    return img, norm_img

if __name__ == '__main__':
    
    #args = parser.parse_args()
    #Here we insert our own bootlegged arguments list
    #
    args = parser.parse_args(['--checkpoint=data/model_checkpoint.pt','--img='+img_path])
    #
    #
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # Load pretrained model
    model = hmr(config.SMPL_MEAN_PARAMS).to(device)
    checkpoint = torch.load(args.checkpoint)
    model.load_state_dict(checkpoint['model'], strict=False)

    # Load SMPL model
    smpl = SMPL(config.SMPL_MODEL_DIR,
                batch_size=1,
                create_transl=False).to(device)
    model.eval()

    # Setup renderer for visualization
    renderer = Renderer(focal_length=constants.FOCAL_LENGTH, img_res=constants.IMG_RES, faces=smpl.faces)


    # Preprocess input image and generate predictions
    img, norm_img = process_image(args.img, args.bbox, args.openpose, input_res=constants.IMG_RES)
    with torch.no_grad():
        pred_rotmat, pred_betas, pred_camera = model(norm_img.to(device))
        pred_output = smpl(betas=pred_betas, body_pose=pred_rotmat[:,1:], global_orient=pred_rotmat[:,0].unsqueeze(1), pose2rot=False)
        pred_vertices = pred_output.vertices
        
    # Calculate camera parameters for rendering
    camera_translation = torch.stack([pred_camera[:,1], pred_camera[:,2], 2*constants.FOCAL_LENGTH/(constants.IMG_RES * pred_camera[:,0] +1e-9)],dim=-1)
    camera_translation = camera_translation[0].cpu().numpy()
    pred_vertices = pred_vertices[0].cpu().numpy()
    img = img.permute(1,2,0).cpu().numpy()

    
    # Render parametric shape
    img_shape = renderer(pred_vertices, camera_translation, img)
    
    # Render side views
    aroundy = cv2.Rodrigues(np.array([0, np.radians(90.), 0]))[0]
    center = pred_vertices.mean(axis=0)
    rot_vertices = np.dot((pred_vertices - center), aroundy) + center
    
    # Render non-parametric shape
    img_shape_side = renderer(rot_vertices, camera_translation, np.ones_like(img))

    outfile = args.img.split('.')[0] if args.outfile is None else args.outfile

    # Save reconstructions
    cv2.imwrite(outfile + '_shape.png', 255 * img_shape[:,:,::-1])
    cv2.imwrite(outfile + '_shape_side.png', 255 * img_shape_side[:,:,::-1])

In [3]:
# Now we have the pose without the root stored on pred_output.body_pose as an array of rotation matrices

pred_output.body_pose.size()

torch.Size([1, 23, 3, 3])

# From 3dpw readme.txt:


3DPW Dataset
============

The 3DPW dataset contains several motion sequences, which are organized into two folders: imageFiles and sequenceFiles.
The folder imageFiles contains the RGB-images for every sequence. 
The folder sequenceFiles provides synchronized motion data and SMPL model parameters in the form of .pkl-files. For each sequence, the .pkl-file contains a dictionary with the following fields:
- sequence: String containing the sequence name
- betas: SMPL shape parameters for each actor which has been used for tracking (List of 10x1 SMPL beta parameters)
- poses: SMPL body poses for each actor aligned with image data (List of Nx72 SMPL joint angles, N = #frames)
- trans: tranlations for each actor aligned with image data (List of Nx3 root translations)
- poses_60Hz: SMPL body poses for each actor at 60Hz (List of Nx72 SMPL joint angles, N = #frames)
- trans_60Hz: tranlations for each actor at 60Hz (List of Nx3 root translations)
- betas_clothed: SMPL shape parameters for each clothed actor (List of 10x1 SMPL beta parameters)
- v_template_clothed: 
- gender: actor genders (List of strings, either 'm' or 'f')
- texture_maps: texture maps for each actor
- poses2D: 2D joint detections in Coco-Format for each actor (only provided if at least 6 joints were detected correctly)
- jointPositions: 3D joint positions of each actor (List of Nx(24*3) XYZ coordinates of each SMPL joint)
- img_frame_ids: an index-array to down-sample 60 Hz 3D poses to corresponding image frame ids
- cam_poses: camera extrinsics for each image frame (Ix4x4 array, I frames times 4x4 homegenous rigid body motion matrices)
- campose_valid: a boolean index array indicating which camera pose has been aligned to the image
- cam_intrinsics: camera intrinsics (K = [f_x 0 c_x;0 f_y c_y; 0 0 1])

Each sequence has either one or two models, which corresponds to the list size of the model specific fields (e.g. betas, poses, trans, v_template, gender, texture_maps, jointPositions, poses2D). 
SMPL poses and translations are provided at 30 Hz. They are aligned to image dependent data (e.g. 2D poses, camera poses). In addition we provide 'poses_60Hz' and 'trans_60Hz' which corresponds to the recording frequency of 60Hz of the IMUs . You could use the 'img_frame_ids' to downsample and align 60Hz 3D and image dependent data, wich has been done to compute SMPL 'poses' and 'trans' variables. 
Please refer to the demo.py-file for loading a sequence, setup smpl-Models and camera, and to visualize an example frame.

In [4]:
# Secondly, we load the .pkl sequence file containing the ground-truth information from 3dpw

import pickle as pkl
import os

#seq_name = 'courtyard_basketball_01'
#datasetDir = 'data/3dpw'
#file = os.path.join(datasetDir,'sequenceFiles/validation',seq_name+'.pkl')
seq = pkl.load(open(pickle_path,'rb'),encoding='latin-1') # opening the sequence file, latin-1 encoding for making it compatible with python3

In [5]:
#seq['campose_valid']

In [6]:
# Getting the sequence of poses for the same frame

seq_reshaped = np.reshape(seq['poses'][0][frame], (24, -1)) # reshaping the sequence file to make it inputtable on R.from_rotvec

from scipy.spatial.transform import Rotation as R

r = R.from_rotvec(seq_reshaped)
seq_matrix = r.as_dcm()
#seq_matrix

In [7]:
#seq_matrix

In [8]:
seq_tensor = torch.as_tensor(seq_matrix,dtype=torch.float, device='cuda').unsqueeze(0)
seq_tensor.size()
#seq_tensor

torch.Size([1, 24, 3, 3])

In [9]:
#seq['betas']

gt_betas = torch.as_tensor(seq['betas'],dtype=torch.float, device='cuda')

In [10]:
q = R.from_dcm(pred_rotmat.squeeze().cpu())
pred_rotvec = q.as_rotvec()
np.size(pred_rotvec)

72

In [11]:
np.mean(np.degrees(abs(pred_rotvec[1:]-seq_reshaped[1:])))

12.873164311583473

In [12]:
pred_rotvec_reshaped = np.reshape(pred_rotvec, (1,72)).squeeze()

In [13]:
pred_rotvec_reshaped

array([-1.80680941e+00, -4.30742841e-01,  2.50211543e+00, -2.18159790e-01,
       -8.26423701e-04,  1.17072648e-01, -3.46963739e-02, -1.12375434e-01,
       -2.37044785e-01,  7.80402855e-02,  4.33914872e-02,  4.20902264e-02,
        5.90166599e-01, -8.97107394e-02, -1.44614235e-01,  8.58142789e-01,
        1.12419867e-01,  8.54071782e-02, -2.82334541e-02, -8.23451998e-03,
        6.20934527e-02, -8.00017958e-02,  1.50744948e-01, -1.10585561e-01,
       -1.46063239e-01, -2.01474634e-01,  1.87965448e-01,  1.32010577e-02,
        2.98754888e-02,  4.19070636e-02, -1.84603255e-01,  1.18182322e-01,
        2.31062595e-01, -1.64441815e-01,  9.55183146e-02, -2.67810320e-01,
       -9.82190088e-02,  1.35472601e-01,  4.63388091e-02, -2.33408097e-01,
       -3.41959485e-01,  6.48425248e-02, -2.27770596e-01,  4.11493866e-01,
        1.85263069e-02,  1.95638807e-01,  2.47533891e-02, -3.83829527e-02,
       -2.83260367e-01, -5.04542844e-01, -1.63743926e-01, -1.80802842e-01,
        5.76958678e-01,  

In [14]:
seq['poses'][0][frame]

array([-0.28323011, -2.37571864, -0.36773444,  0.05345616,  0.04584329,
        0.17431025,  0.33318455,  0.24418301, -0.18979568,  0.4491481 ,
       -0.0033687 ,  0.09576073,  0.47926164, -0.05526007, -0.0273421 ,
        0.58604834,  0.12546303,  0.03199589, -0.00498974,  0.09450928,
       -0.0929267 , -0.08502203,  0.02919686, -0.1052083 ,  0.13738988,
        0.13089518, -0.08245307, -0.05093639,  0.05650342, -0.05291118,
       -0.00968661, -0.01166829,  0.05093711, -0.04038782, -0.09239512,
        0.06190808, -0.19237811,  0.40910108, -0.5161581 , -0.3134727 ,
       -0.39065501,  0.21351273, -0.17941227,  0.09521016, -0.18272589,
       -0.0361012 , -0.05160391,  0.4271788 , -0.54903978, -0.60670174,
       -0.28590509, -0.62409388,  1.29995059,  0.09936155,  0.40626295,
       -0.75205729,  1.19148363, -0.1055585 ,  1.26133793, -0.28787622,
        0.45677271,  0.07896642,  1.39030024, -0.19417673,  0.06500082,
       -1.05771343, -0.25843519, -0.03490923, -0.30970644, -0.14

In [15]:
differences = np.int_(np.degrees(abs(pred_rotvec_reshaped-seq['poses'][0][frame])))
differences

array([ 87, 111, 164,  15,   2,   3,  21,  20,   2,  21,   2,   3,   6,
         1,   6,  15,   0,   3,   1,   5,   8,   0,   6,   0,  16,  19,
        15,   3,   1,   5,  10,   7,  10,   7,  10,  18,   5,  15,  32,
         4,   2,   8,   2,  18,  11,  13,   4,  26,  15,   5,   6,  25,
        41,  10,  34,  35,  38,   4,  15,  17,  24,  13,  71,   9,   5,
        54,  11,   0,   8,   5,   1,   8])

In [16]:
#np.mean(np.degrees(abs(pred_rotvec_reshaped[3:]-np.flip(seq['poses'][0][frame][3:]))))

In [17]:
pred_rotmat.size()

torch.Size([1, 24, 3, 3])

In [18]:
seq_tensor.size()

torch.Size([1, 24, 3, 3])

In [19]:
# Running the demo code again, but this time with the ground-truth pose from 3dpw

In [20]:
"""
Demo code

To run our method, you need a bounding box around the person. The person needs to be centered inside the bounding box and the bounding box should be relatively tight. You can either supply the bounding box directly or provide an [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) detection file. In the latter case we infer the bounding box from the detections.

In summary, we provide 3 different ways to use our demo code and models:
1. Provide only an input image (using ```--img```), in which case it is assumed that it is already cropped with the person centered in the image.
2. Provide an input image as before, together with the OpenPose detection .json (using ```--openpose```). Our code will use the detections to compute the bounding box and crop the image.
3. Provide an image and a bounding box (using ```--bbox```). The expected format for the json file can be seen in ```examples/im1010_bbox.json```.

Example with OpenPose detection .json
```
python3 demo.py --checkpoint=data/model_checkpoint.pt --img=examples/im1010.png --openpose=examples/im1010_openpose.json
```
Example with predefined Bounding Box
```
python3 demo.py --checkpoint=data/model_checkpoint.pt --img=examples/im1010.png --bbox=examples/im1010_bbox.json
```
Example with cropped and centered image
```
python3 demo.py --checkpoint=data/model_checkpoint.pt --img=examples/im1010.png
```

Running the previous command will save the results in ```examples/im1010_{shape,shape_side}.png```. The file ```im1010_shape.png``` shows the overlayed reconstruction of human shape. We also render a side view, saved in ```im1010_shape_side.png```.
"""

import torch
from torchvision.transforms import Normalize
import numpy as np
import cv2
import argparse
import json

from models import hmr, SMPL
from utils.imutils import crop
from utils.renderer_gt import Renderer
import config
import constants

parser = argparse.ArgumentParser()
parser.add_argument('--checkpoint', required=True, help='Path to pretrained checkpoint')
parser.add_argument('--img', type=str, required=True, help='Path to input image')
parser.add_argument('--bbox', type=str, default=None, help='Path to .json file containing bounding box coordinates')
parser.add_argument('--openpose', type=str, default=None, help='Path to .json containing openpose detections')
parser.add_argument('--outfile', type=str, default=None, help='Filename of output images. If not set use input filename.')

def bbox_from_openpose(openpose_file, rescale=1.2, detection_thresh=0.2):
    """Get center and scale for bounding box from openpose detections."""
    with open(openpose_file, 'r') as f:
        keypoints = json.load(f)['people'][0]['pose_keypoints_2d']
    keypoints = np.reshape(np.array(keypoints), (-1,3))
    valid = keypoints[:,-1] > detection_thresh
    valid_keypoints = keypoints[valid][:,:-1]
    center = valid_keypoints.mean(axis=0)
    bbox_size = (valid_keypoints.max(axis=0) - valid_keypoints.min(axis=0)).max()
    # adjust bounding box tightness
    scale = bbox_size / 200.0
    scale *= rescale
    return center, scale

def bbox_from_json(bbox_file):
    """Get center and scale of bounding box from bounding box annotations.
    The expected format is [top_left(x), top_left(y), width, height].
    """
    with open(bbox_file, 'r') as f:
        bbox = np.array(json.load(f)['bbox']).astype(np.float32)
    ul_corner = bbox[:2]
    center = ul_corner + 0.5 * bbox[2:]
    width = max(bbox[2], bbox[3])
    scale = width / 200.0
    # make sure the bounding box is rectangular
    return center, scale

def process_image(img_file, bbox_file, openpose_file, input_res=224):
    """Read image, do preprocessing and possibly crop it according to the bounding box.
    If there are bounding box annotations, use them to crop the image.
    If no bounding box is specified but openpose detections are available, use them to get the bounding box.
    """
    normalize_img = Normalize(mean=constants.IMG_NORM_MEAN, std=constants.IMG_NORM_STD)
    img = cv2.imread(img_file)[:,:,::-1].copy() # PyTorch does not support negative stride at the moment
    if bbox_file is None and openpose_file is None:
        # Assume that the person is centerered in the image
        height = img.shape[0]
        width = img.shape[1]
        center = np.array([width // 2, height // 2])
        scale = max(height, width) / 200
    else:
        if bbox_file is not None:
            center, scale = bbox_from_json(bbox_file)
        elif openpose_file is not None:
            center, scale = bbox_from_openpose(openpose_file)
    img = crop(img, center, scale, (input_res, input_res))
    img = img.astype(np.float32) / 255.
    img = torch.from_numpy(img).permute(2,0,1)
    norm_img = normalize_img(img.clone())[None]
    return img, norm_img

if __name__ == '__main__':
    
    #args = parser.parse_args()
    #Here we insert our own bootlegged arguments list
    #
    args = parser.parse_args(['--checkpoint=data/model_checkpoint.pt','--img='+img_path])
    #
    #
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # Load pretrained model
    model = hmr(config.SMPL_MEAN_PARAMS).to(device)
    checkpoint = torch.load(args.checkpoint)
    model.load_state_dict(checkpoint['model'], strict=False)

    # Load SMPL model
    smpl = SMPL(config.SMPL_MODEL_DIR,
                batch_size=1,
                create_transl=False).to(device)
    model.eval()

    # Setup renderer for visualization
    renderer = Renderer(focal_length=constants.FOCAL_LENGTH, img_res=constants.IMG_RES, faces=smpl.faces)


    # Preprocess input image and generate predictions
    img, norm_img = process_image(args.img, args.bbox, args.openpose, input_res=constants.IMG_RES)
    with torch.no_grad():
        pred_rotmat, pred_betas, pred_camera = model(norm_img.to(device))
        
        # We are bootlegging our ground_truth parameters here
        pred_output = smpl(betas=gt_betas, body_pose=seq_tensor[:,1:], global_orient=seq_tensor[:,0].unsqueeze(1), pose2rot=False)
        #
        
        pred_vertices = pred_output.vertices
        
    # Calculate camera parameters for rendering
    camera_translation = torch.stack([pred_camera[:,1], pred_camera[:,2], 2*constants.FOCAL_LENGTH/(constants.IMG_RES * pred_camera[:,0] +1e-9)],dim=-1)
    camera_translation = camera_translation[0].cpu().numpy()
    pred_vertices = pred_vertices[0].cpu().numpy()
    img = img.permute(1,2,0).cpu().numpy()

    
    # Render parametric shape
    img_shape = renderer(pred_vertices, camera_translation, img, pickle_path, frame)
    
    # Render side views
    aroundy = cv2.Rodrigues(np.array([0, np.radians(90.), 0]))[0]
    center = pred_vertices.mean(axis=0)
    rot_vertices = np.dot((pred_vertices - center), aroundy) + center
    
    # Render non-parametric shape
    img_shape_side = renderer(rot_vertices, camera_translation, np.ones_like(img), pickle_path, frame)

    outfile = args.img.split('.')[0] if args.outfile is None else args.outfile
    
    # Here we add _gt_ to differentiate the output

    # Save reconstructions
    cv2.imwrite(outfile + '_gt_shape.png', 255 * img_shape[:,:,::-1])
    cv2.imwrite(outfile + '_gt_shape_side.png', 255 * img_shape_side[:,:,::-1])

In [21]:
pred_vertices

array([[-0.02610572,  0.47167838,  0.08869465],
       [-0.03288239,  0.4595586 ,  0.07986948],
       [-0.02875192,  0.45305082,  0.09225396],
       ...,
       [ 0.08294842,  0.39774483, -0.01399018],
       [ 0.08200231,  0.399822  , -0.01449837],
       [ 0.0796793 ,  0.39826077, -0.01800945]], dtype=float32)

In [22]:
camera_translation = torch.stack([pred_camera[:,1], pred_camera[:,2], 2*constants.FOCAL_LENGTH/(constants.IMG_RES * pred_camera[:,0] +1e-9)],dim=-1)

In [23]:
camera_translation

tensor([[2.0216e-01, 1.7621e-02, 6.3130e+01]], device='cuda:0')

In [24]:
pred_camera

tensor([[0.7072, 0.2022, 0.0176]], device='cuda:0')

In [25]:
seq['cam_poses'][502]

array([[ 0.87993836,  0.01627723,  0.47480895, -4.51536851],
       [-0.12350747, -0.95721291,  0.26170471, -1.09855333],
       [ 0.45875308, -0.28892647, -0.84027799,  1.71583693],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [26]:
seq['cam_poses'][frame][0:3,0:3]

array([[ 0.87993836,  0.01627723,  0.47480895],
       [-0.12350747, -0.95721291,  0.26170471],
       [ 0.45875308, -0.28892647, -0.84027799]])

In [27]:
res_factor = (seq['cam_intrinsics'][0,2]/(constants.IMG_RES/2))
seq['cam_intrinsics'][1,2]/res_factor
# cam_intrinsics: camera intrinsics (K = [f_x 0 c_x;0 f_y c_y; 0 0 1])

199.11111111111111

In [28]:
224 // 2

112

In [29]:
seq['cam_intrinsics']

array([[1.96185286e+03, 0.00000000e+00, 5.40000000e+02],
       [0.00000000e+00, 1.96923077e+03, 9.60000000e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [30]:
seq_tensor[:,0]

tensor([[[-0.7272,  0.3014, -0.6167],
         [ 0.1008,  0.9356,  0.3383],
         [ 0.6790,  0.1839, -0.7108]]], device='cuda:0')

In [31]:
pred_rotmat[:,0]

tensor([[[-0.3274,  0.1399, -0.9345],
         [ 0.1807, -0.9615, -0.2072],
         [-0.9274, -0.2367,  0.2895]]], device='cuda:0')

In [32]:
pred_minus_gt = pred_rotmat - seq_tensor

In [33]:
deg_tensor = torch.as_tensor(np.degrees(pred_minus_gt.cpu()),dtype=torch.int, device='cuda')